In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

24/08/24 09:11:49 WARN Utils: Your hostname, Eduardos-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.15.33 instead (on interface en0)
24/08/24 09:11:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/24 09:11:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.read.csv('dados/pnad-covid-2020-08.csv.gz').show()

24/08/24 09:11:51 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+---+-------+-------+-----+-----+-----+-----+-------+---------+-----+-----+------+------------+------------+------+----+-----+------+------+------+----+----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+----+----+----+----+----+-----+-----+-----+----+----+-----+-----+-----+-----+-----+------+------+-----+----+----+-----+----+-----+------+--------+-----+------+------+-----+-----+-----+------+-------+--------+------+-------+-------+-----+-----+-----+-----+-----+-----+-----+--------+-----+-----+-----+-----+-----+--------+-----+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+-----+-----+
| _c0|_c1|    _c2|    _c3|  _c4|  _c5|  _c6|  _c7|    _c8|      _c9| _c10| _c11|  _c12|   